In [274]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [275]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [276]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

In [277]:
data.head()

In [278]:
pd.DataFrame(data.isna().sum()).head(10)

## change nan in Alley to no_alley

In [279]:
data['Alley'].unique()

In [280]:
data['Alley'].value_counts()

In [281]:
data['Alley'].fillna("no_alley", inplace=True)

In [282]:
data['Alley'].value_counts()

## Lot  frontage column

In [283]:
data['LotFrontage'].isnull().sum()

In [284]:
# length of the whole data is :
print("length of whole data is :",len(data))

In [285]:
# 259 null values out of 1460 is significant
data['LotFrontage'].value_counts()

In [286]:
cat_cols= data.select_dtypes(include='object').columns
cat_cols

In [287]:
num_cols=data.select_dtypes(exclude='object').columns
num_cols

In [288]:
data[cat_cols].isna().sum()

In [289]:
#drop the MiscFeature, Fence, PoolQC, FireplaceQu columns from the data, as they have very high number of null values

data.drop(['MiscFeature','Fence','PoolQC','FireplaceQu'], axis=1, inplace=True)

In [290]:
# fill the garage columns with nan values - "no_garage"
data['GarageType'].fillna("no_garage", inplace=True)
data['GarageFinish'].fillna("no_garage", inplace=True)
data['GarageQual'].fillna("no_garage", inplace=True)
data['GarageCond'].fillna("no_garage", inplace=True)


In [291]:
# fill the basement columns Nan values with "no_basement"
data['BsmtQual'].fillna("no_basement", inplace=True)
data['BsmtCond'].fillna("no_basement", inplace=True)
data['BsmtExposure'].fillna("no_basement", inplace=True)
data['BsmtFinType1'].fillna("no_basement", inplace=True)
data['BsmtFinType2'].fillna("no_basement", inplace=True)


In [292]:
cat_cols=data.select_dtypes(include="object").columns
data[cat_cols].isna().sum()

In [293]:
cat_cols= data.select_dtypes(include='object').columns
cat_cols

In [294]:
data[num_cols].isnull().sum()

In [295]:
data.corr()['SalePrice'].sort_values()

## garageyrblt column

In [296]:
data['GarageYrBlt'].value_counts()

In [297]:
# drop the null values in the column
data.dropna(subset=['GarageYrBlt'], inplace=True)


## LotFrontage column

In [298]:
data['LotFrontage'].value_counts()

In [299]:
# use KNNImputer for LotFrontage
from sklearn.impute import KNNImputer
imputer =KNNImputer(n_neighbors=5)

In [300]:
data['LotFrontage']=imputer.fit_transform(data[['LotFrontage']])

In [301]:
data[num_cols].isna().sum()

## pipelines

In [302]:
num_cols=['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold','SalePrice']


In [303]:
data=data.drop('Id', axis=1)

In [304]:
num_cols=data.select_dtypes(exclude='object').columns
num_cols

In [305]:
data.select_dtypes(exclude='object').dtypes

In [306]:
cat_cols= data.select_dtypes(include='object').columns
cat_cols

In [307]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler,MaxAbsScaler
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer

col_trans=ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'),cat_cols)])

pipe=Pipeline(steps=[('col_trans', col_trans),('scaler',MaxAbsScaler()),('lin_reg',LinearRegression())])

In [308]:
data=data.dropna(subset=['Electrical'])

In [309]:
data=data.dropna(how='all')

In [310]:
data=data.dropna()

In [311]:
pd.DataFrame(data.isna().any())[20:]

In [312]:
from sklearn.model_selection import train_test_split

x=data.drop(["SalePrice"], axis=1)
y=data['SalePrice']

x_train,x_test, y_train, y_test = train_test_split(x,y, random_state=0, test_size=0.2)

In [313]:
pipe.fit(x_train, y_train)


In [314]:
predictions= pipe.predict(x_test)

## metrics

In [315]:
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

print("mean_squared_error is :", mean_squared_error(y_test, predictions))
print("root_mean_squared_error is :", mean_squared_error(y_test, predictions)**0.5)
print("mean of SalePrice is:", data['SalePrice'].mean())
print("mean_absolue_error is :", mean_absolute_error(y_test, predictions))

## RandomForest

In [316]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler,MaxAbsScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer

col_trans=ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'),cat_cols)])

pipe=Pipeline(steps=[('col_trans', col_trans),('scaler',MaxAbsScaler()),('rf',RandomForestClassifier())])

In [317]:
pipe.fit(x_train,y_train)
predictions= pipe.predict(x_test)

In [318]:
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

print("mean_squared_error is :", mean_squared_error(y_test, predictions))
print("root_mean_squared_error is :", mean_squared_error(y_test, predictions)**0.5)
print("mean of SalePrice is:", data['SalePrice'].mean())
print("mean_absolue_error is :", mean_absolute_error(y_test, predictions))

## Gridsearch and Randomsearch for random forest

In [319]:
#count = {k:0 for k in set(y)}
#for yy in y:
    #count[yy] += 1
#sorted(count.items(), key=lambda i: i[1]) 

In [320]:
# data['SalePrice'].value_counts()

## DecisionTreeClassifier

In [321]:
from sklearn.tree import DecisionTreeClassifier

col_trans=ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'),cat_cols)])

pipe=Pipeline(steps=[('col_trans', col_trans),('scaler',MaxAbsScaler()),('dt',DecisionTreeClassifier())])

In [322]:
pipe.fit(x_train, y_train)
predictions= pipe.predict(x_test)

In [323]:
print("mean_squared_error is :", mean_squared_error(y_test, predictions))
print("root_mean_squared_error is :", mean_squared_error(y_test, predictions)**0.5)
print("mean of SalePrice is:", data['SalePrice'].mean())
print("mean_absolue_error is :", mean_absolute_error(y_test, predictions))

## random forest regression

In [324]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

col_trans=ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'),cat_cols)])

pipe=Pipeline(steps=[('col_trans', col_trans),('scaler',MaxAbsScaler()),('rfr',RandomForestRegressor())])


In [325]:
pipe.fit(x_train,y_train)
predictions= pipe.predict(x_test)

In [326]:
#min_samples_split=[2,5,7,10,12,15]
#min_samples_leaf=[1,2,5,10]
#max_depth=[2,5,8,10,15,20]
#max_features=['auto','sqrt']
#n_estimators=[100,200,300,400,500,600]

#param_grid={'rfr__n_estimators':n_estimators, 'rfr__max_features':max_features, 'rfr__max_depth':max_depth, 
 #           'rfr__min_samples_leaf':min_samples_leaf, 'rfr__min_samples_split': min_samples_split}

#grid=GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=-1 )
#grid.fit(x_train,y_train)


In [327]:
print("mean_squared_error is :", mean_squared_error(y_test, predictions))
print("root_mean_squared_error is :", mean_squared_error(y_test, predictions)**0.5)
print("mean of SalePrice is:", data['SalePrice'].mean())
print("mean_absolue_error is :", mean_absolute_error(y_test, predictions))

## Test dataset

In [328]:
data2= pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
data2.head()

In [329]:
data2.drop(['MiscFeature','Fence','PoolQC','FireplaceQu'], axis=1, inplace=True)
data2['GarageType'].fillna("no_garage", inplace=True)
data2['GarageFinish'].fillna("no_garage", inplace=True)
data2['GarageQual'].fillna("no_garage", inplace=True)
data2['GarageCond'].fillna("no_garage", inplace=True)

data2['BsmtQual'].fillna("no_basement", inplace=True)
data2['BsmtCond'].fillna("no_basement", inplace=True)
data2['BsmtExposure'].fillna("no_basement", inplace=True)
data2['BsmtFinType1'].fillna("no_basement", inplace=True)
data2['BsmtFinType2'].fillna("no_basement", inplace=True)

data2['Alley'].fillna("no_alley", inplace=True)

data2.dropna(subset=['GarageYrBlt'], inplace=True)

from sklearn.impute import KNNImputer
imputer =KNNImputer(n_neighbors=5)
data2['LotFrontage']=imputer.fit_transform(data2[['LotFrontage']])

In [330]:
data2.isna().sum()

In [331]:
nulls1= pd.DataFrame(data2.isna().sum())


In [332]:
data2.shape

In [333]:
data3= data2.dropna(axis=0)
data3.shape

In [334]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

col_trans=ColumnTransformer([('ohe',OneHotEncoder(handle_unknown='ignore'),cat_cols)])

pipe=Pipeline(steps=[('col_trans', col_trans),('scaler',MaxAbsScaler()),('rfr',RandomForestRegressor())])

pipe.fit(x_train, y_train)

In [335]:
data_new= pd.DataFrame(data3, columns= x_train.columns)

In [336]:
data_new.shape

In [337]:
x_train.shape

In [338]:
predicted_values= pipe.predict(data_new)

## My submission

In [339]:
my_submission = pd.DataFrame({'Id': data3['Id'],'SalePrice': predicted_values})
my_submission.head()

In [340]:
my_submission.to_csv('submission_raghu.csv', index=False)